# Spotify Track Analytics Popularity Prediction

Dataset Content
The data set used for this project: [Kaggle](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset). The collection of ~114,000 songs across 125 genres with features like danceability, energy, tempo, and popularity. Ideal for audio analysis, genre classification, and music trend exploration.

The dataset consists of the following columns:

* track_id: Unique Spotify identifier for each track.
* artists: List of artists performing the track, separated by semicolons.
* album_name: Title of the album where the track appears.
* track_name: Title of the song.
* popularity: Score from 0–100 based on recent play counts; higher means more popular.
* duration_ms: Length of the track in milliseconds.
* explicit: Indicates whether the track contains explicit content (True/False).
* danceability: Score (0.0–1.0) measuring how suitable the song is for dancing.
* energy: Score (0.0–1.0) reflecting intensity, speed, and loudness.
* key: Musical key using Pitch Class notation (0 = C, 1 = C♯/D♭, etc.).
* loudness: Overall volume of the track in decibels.
* mode: Indicates scale type (1 = major, 0 = minor).
* speechiness: Score estimating spoken content in the track.
* cousticness: Likelihood (0.0–1.0) that the song is acoustic.
* instrumentalness: Probability that the track has no vocals.
* liveness: Measures if the song was recorded live (higher = more live).
* valence: Positivity of the music (0.0 = sad, 1.0 = happy).
* tempo: Speed of the song in beats per minute (BPM). time_signature: Musical meter (e.g. 4 = 4/4 time). * track_genre: Musical genre classification of the track.

Import libraries that will be used

In [ ]:

from rapidfuzz import process, fuzz  #import RapidFuzz for fuzzy string matching
import pandas as pd                 #import Pandas for data manipulation
import numpy as np                  #import Numpy for numerical operations
import matplotlib.pyplot as plt     #import Matplotlib for data visualization
import seaborn as sns               #import Seaborn for statistical data visualization
from plotly.subplots import make_subplots  #import Plotly subplots for creating complex figures
import plotly.express as px         #import Plotly Express for interactive visualizations

In [69]:
sns.set(style="whitegrid")                  # Set Seaborn style for plots
plt.rcParams["figure.figsize"] = (10,6)     # Set default figure size for Matplotlib plots

## 1. Explanatory Data Analysis

In this section EDA, including data load and cleaning, is performed.
As a first step, data set is loaded into DataFrame

In [70]:
df = pd.read_csv('data/spotify_dataset.csv')  # Load the car price dataset
df.head()                                            # Display the first few rows of the dataset

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


### 1.1.Initial data exploration

In the following subsection initial data set inspection is performed. Here the shape and Info of DataFrame are shown

In [71]:
print(df.shape)                     # Print the shape of the DataFrame           
print(df.info())                    # Print concise summary of the DataFrame            
print(df.dtypes)                    # Print data types of each column

(114000, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  livenes

This dataset contains of 114 entires and 21 rows

In the next steps DataFrame is checked for any incosistencies(dublicates, missing value and etc.)

In [72]:
df.isnull().sum()                  # Check for missing values in each column

Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

As it is can be seen, there are 3 empty etries. This number is neglectable, so we can drop these entries.

In [73]:
df. dropna(inplace=True)          # Drop rows with missing values
df.reset_index(drop=True, inplace=True)  # Reset index after dropping rows

and veryfying, that empty values are removed

In [74]:
df.isnull().sum()                  # Verify that there are no missing values left

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

and check for duplicates

In [75]:
df.duplicated() # Check for duplicate rows in the DataFrame


0         False
1         False
2         False
3         False
4         False
          ...  
113994    False
113995    False
113996    False
113997    False
113998    False
Length: 113999, dtype: bool

here column names are displayed

In [76]:
df.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

here we see that we have column "Unnamed:0" and "track_id". These two columns are not important for further analysis and modeling, therefore they can be dropped

In [77]:
df.drop(columns = ['Unnamed: 0', 'track_id'], inplace=True)  # Drop unnecessary columns and display the first few rows 

In [78]:
df.columns

Index(['artists', 'album_name', 'track_name', 'popularity', 'duration_ms',
       'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')

The next step is check, wether column categorical columns have misspelling in names and, as a result duplicates did not detected.

In [79]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("Categorical columns:")
print(categorical_cols)

Categorical columns:
['artists', 'album_name', 'track_name', 'track_genre']


let's make all names uniqied

In [80]:
# Loop through each categorical column and clean it
for col in categorical_cols:
    df[col] = (
        df[col]
        .astype(str)                 # ensure the column is string type
        .str.strip()                 # remove leading/trailing spaces
        .str.lower()                 # make everything lowercase
        .str.replace("-", " ", regex=False)   # replace hyphens with spaces
        .str.replace("_", " ", regex=False)   # replace underscores with spaces
        .str.replace(";", " and ", regex=False)   # replace semicolons with spaces
    )

# Check the result
df.head()

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,gen hoshino,comedy,comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,ben woodward,ghost (acoustic),ghost acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,ingrid michaelson and zayn,to begin again,to begin again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,kina grannis,crazy rich asians (original motion picture sou...,can't help falling in love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,chord overstreet,hold on,hold on,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


Detecting misspellings or inconsistent categories in categorical columns is super important for EDA and modeling (e.g., "hip hop" vs "Hip-Hop" vs "Hip hop").

In [ ]:
for col in categorical_cols:      # Loop through each categorical column
    print(f"\n🔹 Column: {col}")                   
    print(f"Unique values: {df[col].nunique()}")  # Print number of unique categories
    print(df[col].value_counts())  # Show top 10 most common categories
    print("-" * 50)


🔹 Column: artists
Unique values: 31427
artists
the beatles                                                     279
george jones                                                    271
stevie wonder                                                   236
linkin park                                                     224
ella fitzgerald                                                 222
                                                               ... 
felipe vilela and fernandinho beat box and ericka nascimento      1
paulo rogerio and wandra scott                                    1
leidy murilho and talita barreto and asaph                        1
felipe vilela and gabriela gomes                                  1
jesus culture                                                     1
Name: count, Length: 31427, dtype: int64
--------------------------------------------------

🔹 Column: album_name
Unique values: 46150
album_name
alternative christmas 2022                 195
feliz cumpl

This gives you an overview of what’s inside each categorical column — letting you visually spot potential misspellings.

**rapidfuzz** is used to find and unify them automatically.
It prints potential duplicates that might be typos, case, or spacing differences.

In [ ]:

unique_artists = df['artists'].unique()   # Get unique artist names
artist_map = {}                    # Initialize a mapping dictionary

for artist in unique_artists:  # Loop through each unique artist
    match = process.extractOne(artist, unique_artists, scorer=fuzz.token_sort_ratio)  # Find the closest match
    if match and match[1] > 95 and artist != match[0]:       # If a close match is found
        artist_map[artist] = match[0]   # Map to the closest match

# Apply fuzzy corrections
df['artists'] = df['artists'].replace(artist_map)     # Replace artist names based on the mapping

and verifying changes

In [83]:
print(df["artists"]. unique())

['gen hoshino' 'ben woodward' 'ingrid michaelson and zayn' ...
 'cuencos tibetanos sonidos relajantes'
 'bryan & katie torwalt and brock human' 'jesus culture']


It is also nessecary to convert song duration from milliseconds to mintues and drop "duration_ms" column.

In [84]:
df["duration_min"] = df["duration_ms"] / 60000  # Convert duration from milliseconds to minutes
df.drop(columns=["duration_ms"], inplace=True)  # Drop the original duration_ms column
df.head()  # Display the first few rows to verify changes

,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_min
0,gen hoshino,comedy,comedy,73,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,3.844433
1,ben woodward,ghost (acoustic),ghost acoustic,55,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,2.493500
2,ingrid michaelson and zayn,to begin again,to begin again,57,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,3.513767
3,kina grannis,crazy rich asians (original motion picture sou...,can't help falling in love,71,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,3.365550
4,chord overstreet,hold on,hold on,82,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,3.314217


Next step is to take a look on a descriptive statistic summary for each column in the df.

We use descriptive statistics to summarize, explore, and validate the dataset before modeling. They give us a quick overview of central tendencies, spread, and anomalies — helping us decide how to clean, visualize, and model the data.

This summary includes following metrics:

* count: Number of non-missing (non-NaN) values. Helps check missing data.
* mean: Average (sum / count). Central tendency of numeric data.
* std: Standard deviation. How spread out the values are from the mean.
* min: Minimum value. The smallest observed value.
* 25%: 25th percentile (Q1). 25% of data is below this value
* 50% (median): 50th percentile. Half the data is below this value.
* 75%: 75th percentile (Q3). 75% of data is below this value
* max: Maximum value. The largest observed value
In following cell a descriptive statistics of numeric columns is performed:

In [85]:
df.describe() # Generate descriptive statistics of numerical columns

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_min
count,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000
mean,33.238827,0.566801,0.641383,5.309126,-8.258950,0.637558,0.084652,0.314907,0.156051,0.213554,0.474066,122.147695,3.904034,3.800519
std,22.304959,0.173543,0.251530,3.559999,5.029357,0.480708,0.105733,0.332522,0.309556,0.190378,0.259261,29.978290,0.432623,1.788268
min,0.000000,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143100
25%,17.000000,0.456000,0.472000,2.000000,-10.013000,0.000000,0.035900,0.016900,0.000000,0.098000,0.260000,99.218500,4.000000,2.901100
50%,35.000000,0.580000,0.685000,5.000000,-7.004000,1.000000,0.048900,0.169000,0.000042,0.132000,0.464000,122.017000,4.000000,3.548433
75%,50.000000,0.695000,0.854000,8.000000,-5.003000,1.000000,0.084500,0.597500,0.049000,0.273000,0.683000,140.071000,4.000000,4.358433
max,100.000000,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000,87.288250


The dataset contains 113,999 tracks, and the statistics cover 14 numerical features related to track popularity and audio characteristics.

Key Feature Insights
1. Popularity
Range: 0 to 100 (Spotify-defined scale). Mean: 33.2 (relatively low), Median: 35.
Insight: Majority of songs in the dataset are not highly popular. Only a small portion reaches scores above 75.
2. Danceability
Scale: 0.0 to 1.0 (higher = more danceable). Mean: 0.567.
Insight: Most tracks are moderately danceable. Distribution is slightly right-skewed with many songs in the 0.5–0.7 range.
3. Energy
Scale: 0.0 to 1.0. Mean: 0.64.
Insight: Tracks generally have high energy, indicating a tendency toward upbeat or intense music.
4. Key
Range: 0 to 11 (12 semitones, C to B). Mean: ~5.3.
Insight: Keys are evenly distributed, with slight clustering near 5 (F major / D minor).
5. Loudness
Unit: Decibels (dB). Mean: -8.26 dB, Min: -49.5 dB.
Insight: Some tracks have extremely low loudness, possibly ambient or silent tracks; majority are mastered for streaming loudness levels (around -7 dB).
6. Mode
Binary: 0 = minor, 1 = major. Mean: 0.637 → ~64% of songs are in major mode.
Insight: Major mode dominates (typically associated with “happy” sound).
7. Speechiness
Scale: 0.0 to 1.0. Mean: 0.084.
Insight: Most tracks have low speech content (e.g., songs, not podcasts), but the max of 0.965 suggests some spoken word/music hybrids.
8. Acousticness
Mean: 0.315. Insight: Majority of tracks are not acoustic-heavy, but the high standard deviation (0.33) shows some variety.
9. Instrumentalness
Mean: 0.156. 
Insight: Most tracks contain vocals (median ~0.000042), but there’s a small but significant subset of instrumental music.
10. Liveness
Mean: 0.213.
Insight: Most tracks are studio recordings; live recordings are rare.
11. Valence
Scale: 0.0 (sad) to 1.0 (happy).
Mean: 0.47.
Insight: Balanced distribution between positive and negative mood songs.
12. Tempo
Mean: 122 BPM. 
Insight: Common tempo for pop/dance tracks. Range is wide (0–243 BPM), but the quartiles (25% = 99 BPM, 75% = 140 BPM) confirm a core BPM range of ~100–140.
13. Time Signature
Mean: ~3.9. 
Insight: Most tracks are in 4/4 time (common time), as expected. Very little variation.
14. Duration (in minutes)
Mean: 3.8 minutes. 
Max: 87 minutes!
Insight: Typical track duration matches mainstream standards. Max suggests presence of podcasts, live sets, or compilation tracks.

In [87]:
df_cleaned = df.copy()  # Create a copy of the cleaned DataFrame
df_cleaned.to_csv('data/spotify_cleaned_data.csv', index=False)

### 1.2 Initial data visualization

In this section initial data visualisation is performed